# Test memory usage for Shiqi
---

In [1]:
import matplotlib.pyplot as plt
import numpy             as np
import torch

from astropy                import units, constants

from pomme.plot          import plot_cube_2D, plot_spectrum
from pomme.model         import TensorModel
from pomme.lines         import Line
from pomme.haar          import Haar

import magritte.core as magritte

In [2]:
line = Line(
        species_name = "co",
        transition   = 0,
        datafile     = "data/co.txt",
        molar_mass   = 28.0
    )

You have selected line:
    co(J=1-0)
Please check the properties that were inferred:
    Frequency         1.152712018e+11  Hz
    Einstein A coeff  7.203000000e-08  1/s
    Molar mass        28.0             g/mol


In [3]:
model         = magritte.Model("/STER/frederikd/0766.hdf5")
model.compute_spectral_discretisation ()
model.compute_inverse_line_widths     ()
model.compute_LTE_level_populations   ()

fcen = model.lines.lineProducingSpecies[0].linedata.frequency[0]
vpix = 300   # velocity pixel size [m/s] 
dd   = vpix * (model.parameters.nfreqs()-1)/2 / magritte.CC
fmin = fcen - fcen*dd
fmax = fcen + fcen*dd

                                           
-------------------------------------------
  Reading Model...                         
-------------------------------------------
 model file = /STER/frederikd/0766.hdf5
-------------------------------------------
Reading parameters...
Reading points...
Reading rays...
Reading boundary...
Reading chemistry...
Reading species...
Reading thermodynamics...
Reading temperature...
Reading turbulence...
Reading lines...
Reading lineProducingSpecies...
Reading linedata...
read num 1
read sym CO
nlev = 41
nrad = 1
Reading collisionPartner...
Reading collisionPartner...
Reading quadrature...
Reading radiation...
Reading frequencies...
Not using scattering!
                                           
-------------------------------------------
  Model read, parameters:                  
-------------------------------------------
  npoints    = 391168
  nrays      = 12
  nboundary  = 8056
  nfreqs     = 31
  nspecs     = 5
  nlspecs    = 1
  nlines  

In [4]:
# start computing Intensity
model.compute_spectral_discretisation (fmin, fmax)
# along the first ray
model.compute_image (0)
image_nr = -1
frequencies = np.array(model.images[image_nr].freqs)
frequencies = torch.from_numpy(frequencies)

position    = np.array(model.geometry.points.position)        # shape (427008,3)
velocity    = np.array(model.geometry.points.velocity)* constants.c.si.value        # shape (427008,3)
temperature = np.array(model.thermodynamics.temperature.gas)  # shape (427008,)
abundance   = np.array(model.chemistry.species.abundance)
CO          = abundance[:,1]                                  # shape (427008,)
vturb2      = np.array(model.thermodynamics.turbulence.vturb2) # shape (419840,)
vturb = np.sqrt(vturb2)

v_x = velocity[:,0]
v_y = velocity[:,1]
v_z = velocity[:,2]

Computing spectral discretisation...
Computing image...


In [5]:
# input cubes
haar = Haar(position, q=9)
# nCO_dat   = haar.map_data(CO, interpolate=True)[-1][   64:192,64:192,64:192]
# tmp_dat   = haar.map_data(temperature, interpolate=True)[-1][   64:192,64:192,64:192]
# vturb_dat = haar.map_data(vturb, interpolate=True)[-1][   64:192,64:192,64:192]
# v_x_dat   = haar.map_data(v_x, interpolate=True)[-1][   64:192,64:192,64:192]
# v_y_dat   = haar.map_data(v_y, interpolate=True)[-1][   64:192,64:192,64:192]
# v_z_dat   = haar.map_data(v_z, interpolate=True)[-1][   64:192,64:192,64:192]
nCO_dat   = haar.map_data(CO, interpolate=True)[-1]
tmp_dat   = haar.map_data(temperature, interpolate=True)[-1]
vturb_dat = haar.map_data(vturb, interpolate=True)[-1]
v_x_dat   = haar.map_data(v_x, interpolate=True)[-1]
v_y_dat   = haar.map_data(v_y, interpolate=True)[-1]
v_z_dat   = haar.map_data(v_z, interpolate=True)[-1]

In [6]:
# creare model
pomme_model = TensorModel(shape=nCO_dat.shape, sizes=haar.xyz_L)
pomme_model['CO'         ]  = nCO_dat
pomme_model['temperature']  = tmp_dat
pomme_model['v_turbulence'] = vturb_dat
pomme_model['velocity_x']       =        v_x_dat
pomme_model['velocity_y']       =        v_y_dat
pomme_model['velocity_z']       =        v_z_dat

In [7]:
with torch.no_grad():
    img = line.LTE_image_along_last_axis(
        density      = pomme_model['CO'         ],
        temperature  = pomme_model['temperature'],
        v_turbulence = pomme_model['v_turbulence'],
        velocity_los = pomme_model['velocity_z'],
        frequencies  = frequencies,
        dx           = pomme_model.dx(3-1)
    )

# Avoid negative values (should probably avoid these earlier...)
img = torch.abs(img)

In [9]:
nCO_dat.shape

(128, 128, 128)

In [10]:
plot_cube_2D(torch.log(img))

interactive(children=(IntSlider(value=15, description='z', max=30), Output()), _dom_classes=('widget-interact'…

<function pomme.plot.plot_cube_2D.<locals>.plot(z)>

In [12]:
plot_spectrum(torch.log(img))

interactive(children=(IntSlider(value=63, description='i', max=127), IntSlider(value=63, description='j', max=…

<function pomme.plot.plot_spectrum.<locals>.plot(i, j)>